In [ ]:
import sys
print(sys.version)

In [ ]:
import tifffile as tif

from skimage import io

import numpy as np
from time import time

import pandas as pd

import csv
import os

from skimage.transform import resize

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

import pandas as pd

In [ ]:
import skimage
print("skimage", skimage.__version__)
## if skimage < 0.15 needs to manually import their resize (if using resize), for anti_aliasing

In [ ]:
print("keras", keras.__version__)
#print("tensorflow", tf.__version__)

In [ ]:
# check the backend the ordering of the channels
print(keras.backend.backend())
print(keras.backend.image_dim_ordering())
print(K.image_data_format())

In [ ]:
import matplotlib.pyplot as plt
plt.set_cmap('Greys');

%matplotlib inline

In [ ]:
input_dir = 'YOURPATH' ## of augmented npy files

In [ ]:
## For testing different parameters:
##  run the notebook with this cell.

## Run the notebook as a parameterized script:

## CUDA_VISIBLE_DEVICES="0" KERAS_BACKEND=tensorflow WING_CNN_CSV="XXXX" jupyter nbconvert --execute --ExecutePreprocessor.timeout=-1 1_CNN_wing_landmark_detection.ipynb


# csv_folder = os.path.join(input_dir,'csv_dir')

# with open(os.path.join(csv_folder,f'8.csv'), newline='') as csvfile:
#     reader = csv.reader(csvfile, delimiter=',')
#     params = {row[0]:row[1] for row in reader}
    
# params

In [ ]:
## Also - only if running parallel - e.g to test different sigmas:

# import tensorflow as tf
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

## Load data

In [ ]:
X_name = 'X.npy'
Y_name = 'Y.npy'

In [ ]:
x = np.load(os.path.join(input_dir, X_name))
y = np.load(os.path.join(input_dir, Y_name))

### Def NN

In [ ]:
n_conv = 64
n_output_channels = y.shape[-1]
dropout_rate = 0.25

batch_size = 4
epochs = 400

In [ ]:
def cnn(input_shape, n_output_channels):

    x_in = Input(input_shape)

    x1 = Conv2D(n_conv, kernel_size=3, padding="same", activation="relu")(x_in)
    x1 = Conv2D(n_conv, kernel_size=3, padding="same", activation="relu")(x1)
    x1 = BatchNormalization()(x1)
    
    x1_pool = MaxPooling2D(pool_size=2, strides=2, padding="same")(x1)
    x1_pool = Dropout(0.25)(x1_pool)
    
    x2 = Conv2D(n_conv*2, kernel_size=3, padding="same", activation="relu")(x1_pool)
    x2 = Conv2D(n_conv*2, kernel_size=3, padding="same", activation="relu")(x2)
    x2 = BatchNormalization()(x2)
    
    x2_pool = MaxPooling2D(pool_size=2, strides=2, padding="same")(x2)
    x2_pool = Dropout(0.25)(x2_pool)

    x3 = Conv2D(n_conv*4, kernel_size=3, padding="same", activation="relu")(x2_pool)
    x3 = Conv2D(n_conv*4, kernel_size=3, padding="same", activation="relu")(x3)
    x3 = BatchNormalization()(x3)

    x4 = Conv2DTranspose(n_conv*2, kernel_size=3, strides=2, padding="same", activation="relu")(x3)
    x4 = Conv2D(n_conv*2, kernel_size=3, padding="same", activation="relu")(x4)
    x4 = BatchNormalization()(x4)
    x4 = Dropout(0.25)(x4)

    x_out = Conv2DTranspose(n_output_channels, kernel_size=3, strides=2, padding="same", activation="linear")(x4)

    # Compile
    CNN = Model(inputs=x_in, outputs=x_out, name="CNN")
    CNN.compile(optimizer=Adam(), loss="mean_squared_error")

    return CNN

### Input, Shuffle, Split Function:

In [ ]:
def load_input_set_shuffle_and_split(x, y, is_shuffle=True, shuffle_seed=0, n_train=20):
    
    if y.ndim == 3:
        y = y.astype('float32')[:,:,:, None]
        
    x = x.astype('float32')[:,:,:, None]

    print(f'x size: {x.shape}')
    print(f'y size: {y.shape}')
                             
    input_shape = x.shape[1:] + (1,)
    
    # Check Y Range:
    print(f'max Y: {np.max(y)}, min Y: {np.min(y)}')
    
    if is_shuffle:
        x, y = shuffle(x, y, random_state=shuffle_seed)

    ## If not square image, pad the smaller dim:
    if input_shape[0]>input_shape[1]:
        pad_size = input_shape[0] - input_shape[1]
        x = np.pad(x, ((0,0),(0,0),(pad_size,0),(0,0)), 'maximum')
        y = np.pad(y, ((0,0),(0,0),(pad_size,0),(0,0)), 'maximum')

    if input_shape[0]<input_shape[1]:
        pad_size = input_shape[1] - input_shape[0]
        x = np.pad(x, ((0,0),(pad_size,0),(0,0),(0,0)), 'maximum')
        y = np.pad(y, ((0,0),(pad_size,0),(0,0),(0,0)), 'maximum')
    
    # Split data to train and test:
    (x_train, y_train) = x[:n_train], y[:n_train]
    (x_test, y_test) = x[n_train:], y[n_train:]

    print(f'y train size: {y_train.shape}')
    print(f'y test size: {y_test.shape}')
    print(f'x train size: {x_train.shape}')
    print(f'x test size: {x_test.shape}')

    # Print x normalization data:
    # Check that x is already normalized:
    
    print(f'mean x_train: {np.mean(x_train)}')
    print(f'std x_train: {np.std(x_train)}')
    print(f'min x_train: {np.min(x_train)}')
    print(f'max x_train: {np.max(x_train)}')
    
    return x_train, x_test, y_train, y_test
    

### Show images - only for notebook

In [ ]:
def show_images(x, y, n_ims_show=5):
    
    # examples of the x images 
    plt.figure()
    plt.rcParams['figure.figsize'] = (15, 15)
    plt.imshow(np.concatenate(x[:n_ims_show,:,:,0],axis=1), interpolation='none')
    plt.axis('off');
    # examples of the y images - first landmark
    y_32 = y.astype(np.float32)
    plt.figure()
    plt.rcParams['figure.figsize'] = (15, 15)
    plt.imshow(np.concatenate(y_32[:n_ims_show,:,:,0],axis=1), interpolation='none')
    plt.axis('off');
    
    # example of overlay all y on x:
    y_overlay = np.max(y_32, axis=3)
    xy_overlay = x[:,:,:,0] + y_overlay
    plt.figure()
    plt.rcParams['figure.figsize'] = (15, 15)
    plt.imshow(np.concatenate(xy_overlay[:n_ims_show,:,:],axis=1), interpolation='none')
    plt.axis('off');

In [ ]:
#def call_model(x_train,x_test,y_train,y_test):
def call_model(train_generator,x_test,y_test):   
    
    input_shape = x_test.shape[1:]
    
    model = cnn(input_shape, y_test.shape[-1])
    model.summary()
    
    # Define folder to save results:
    models_dir = os.path.join(input_dir, 'models')
 
    os.makedirs(models_dir, exist_ok=True)

    # Early stopping, saving checkpoints (only weights), reduce alpha on plateau..
    callbacks = [
    #        EarlyStopping(monitor='val_loss',
    #                               patience=8,
    #                               verbose=1,
    #                               min_delta=1e-4),
    #             ReduceLROnPlateau(monitor='val_loss',
    #                               factor=0.1,
    #                               patience=4,
    #                               verbose=1,
    #                               epsilon=1e-4),
                 ModelCheckpoint(monitor='val_loss',
                                 filepath= os.path.join(models_dir,'{epoch:04d}_{val_loss:.4f}.hdf5'),
                                 period=20,
                                 save_weights_only=True)]

    # Train
    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=callbacks,
              validation_data=(x_test, y_test),
              shuffle=True,
              verbose=2)
    
    return history
    

In [ ]:
# plots the training process
def plot_history(history):
    print("Available data:", history.history.keys())
    # summarize history for accuracy
    
    #plt.figure - needed to draw more than one image in cell
    plt.figure
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()

In [ ]:
def save_history(history):
    #pd.DataFrame(history.history).plot()
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(os.path.join(models_dir,'history.csv'))

In [ ]:
def run_all(x, y, is_show_images=False):
    
    x_train,x_test,y_train,y_test = load_input_set_shuffle_and_split(x, y, is_show_images)

    if is_show_images:
        show_images(x_train, y_train)
        
    history = call_model(x_train,x_test,y_train,y_test)
    

In [ ]:
run_all(x, y, False)

In [ ]:
input_shape = (512,512,1)

In [ ]:
model = cnn(input_shape, 40)
model.summary()